In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Udemy Courses") \
    .getOrCreate()

22/10/20 10:27:24 WARN Utils: Your hostname, 1011000011101110 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlp3s0)
22/10/20 10:27:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 10:27:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("udemy_courses.json")

In [5]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: string (nullable = true)
 |-- num_subscribers: string (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [7]:
df.show(truncate = True)

+---------------+------------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|_corrupt_record|  content_duration|course_id|        course_title|is_paid|             level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+---------------+------------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|           null|               1.5|  1070968|Ultimate Investme...|   True|        All Levels|          51|         23|           2147|  200|2017-01-18T20:58:58Z|Business Finance|https://www.udemy...|
|           null|              39.0|  1113822|Complete GST Cour...|   True|        All Levels|         274|        923|           2792|   75|2017-03-09T16:34:20Z|Business Finance|https://www.udemy

In [10]:
df.toPandas().to_csv('OutputDatas/udemy_courses.csv',index= False)

# 1. What are the best free courses by subject?

In [79]:
free_courses= df.filter(df.is_paid == "False")

In [73]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [80]:
free_courses.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: string (nullable = true)
 |-- num_subscribers: string (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [87]:
free_courses=free_courses.withColumn("num_reviews",col("num_reviews").cast("integer"))
free_courses= free_courses.withColumn("num_subscribers",col("num_subscribers").cast("integer"))

In [88]:
free_courses.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- num_subscribers: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [110]:
free_courses.show(20)

+---------------+------------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|_corrupt_record|  content_duration|course_id|        course_title|is_paid|             level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+---------------+------------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|           null|              0.55|  1148774|Options Trading 1...|  False|    Beginner Level|          11|         66|           1514|    0|2017-03-23T22:19:57Z|Business Finance|https://www.udemy...|
|           null|               1.5|   133536|Stock Market Inve...|  False|    Beginner Level|          15|       2698|          50855|    0|2013-12-25T19:53:34Z|Business Finance|https://www.udemy

In [90]:
best_free_courses = free_courses.orderBy(free_courses.subject,free_courses.num_subscribers.desc(),free_courses.num_reviews.desc())

In [109]:
best_free_courses.show(truncate = True)

+---------------+----------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|_corrupt_record|content_duration|course_id|        course_title|is_paid|             level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+---------------+----------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|           null|             8.0|    49798|Bitcoin or How I ...|  False|        All Levels|          24|        936|          65576|    0|2013-04-20T02:25:22Z|Business Finance|https://www.udemy...|
|           null|             1.5|    48841|Accounting in 60 ...|  False|    Beginner Level|          16|       4397|          56659|    0|2013-04-07T21:39:25Z|Business Finance|https://www.udemy...|
|    

In [117]:
best_free_courses_by_subject=best_free_courses.select(best_free_courses.course_title,best_free_courses.subject,best_free_courses.num_subscribers)

In [118]:
best_free_courses_by_subject.show(truncate=False)
                                

+-----------------------------------------------------------+----------------+---------------+
|course_title                                               |subject         |num_subscribers|
+-----------------------------------------------------------+----------------+---------------+
|Bitcoin or How I Learned to Stop Worrying and Love Crypto  |Business Finance|65576          |
|Accounting in 60 Minutes - A Brief Introduction            |Business Finance|56659          |
|Stock Market Investing for Beginners                       |Business Finance|50855          |
|Introduction to Financial Modeling                         |Business Finance|29167          |
|Forex Basics                                               |Business Finance|22344          |
|Financial Analysis:  A Recipe for Success                  |Business Finance|19614          |
|Stock Market Foundations                                   |Business Finance|19339          |
|Financial Accounting - A Brief Introduction      

In [119]:
best_free_courses_by_subject.toPandas().to_csv('OutputDatas/1.Best_free_courses_by_subject.csv',index= False)

# 2.What are the most popular courses?

In [97]:
most_popular = df

In [98]:
most_popular.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: string (nullable = true)
 |-- num_subscribers: string (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [99]:
most_popular=most_popular.withColumn("num_reviews",col("num_reviews").cast("integer"))
most_popular= most_popular.withColumn("num_subscribers",col("num_subscribers").cast("integer"))

In [100]:
most_popular.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- num_subscribers: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [120]:
most_popular = most_popular.orderBy(most_popular.num_subscribers.desc())

In [123]:
most_popular.show(10)

+---------------+----------------+---------+--------------------+-------+--------------+------------+-----------+---------------+-----+--------------------+-------------------+--------------------+
|_corrupt_record|content_duration|course_id|        course_title|is_paid|         level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|            subject|                 url|
+---------------+----------------+---------+--------------------+-------+--------------+------------+-----------+---------------+-----+--------------------+-------------------+--------------------+
|           null|            10.5|    41295|Learn HTML5 Progr...|  False|    All Levels|          45|       8629|         268923|    0|2013-02-14T07:03:41Z|    Web Development|https://www.udemy...|
|           null|             3.5|    59014|Coding for Entrep...|  False|Beginner Level|          27|        279|         161029|    0|2013-06-09T15:51:55Z|    Web Development|https://www.udemy...|
|         

In [125]:
most_popular_courses = most_popular.select(most_popular.course_title,most_popular.num_subscribers)

In [126]:
most_popular_courses.show(10)

+--------------------+---------------+
|        course_title|num_subscribers|
+--------------------+---------------+
|Learn HTML5 Progr...|         268923|
|Coding for Entrep...|         161029|
|The Web Developer...|         121584|
|Build Your First ...|         120291|
|The Complete Web ...|         114512|
|Free Beginner Ele...|         101154|
|Web Design for We...|          98867|
|Learn Javascript ...|          84897|
|Practical PHP: Ma...|          83737|
|JavaScript: Under...|          79612|
+--------------------+---------------+
only showing top 10 rows



In [127]:
most_popular_courses.toPandas().to_csv('OutputDatas/2.Most_popular_courses.csv',index= False)

# 3. List the courses that are specialized to “Business Finance” and find the average number of subscribers, reviews, price and lectures on the subject.

In [173]:
business_finance =  df.filter(df.subject == "Business Finance")

In [174]:
business_finance.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: string (nullable = true)
 |-- num_reviews: string (nullable = true)
 |-- num_subscribers: string (nullable = true)
 |-- price: string (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [175]:
business_finance = business_finance.withColumn("num_reviews",col("num_reviews").cast("integer"))
business_finance = business_finance.withColumn("num_subscribers",col("num_subscribers").cast("integer"))
business_finance = business_finance.withColumn("num_lectures",col("num_lectures").cast("double"))
business_finance = busines_finance.withColumn("price",col("price").cast("double"))

In [176]:
business_finance.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- content_duration: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- course_title: string (nullable = true)
 |-- is_paid: string (nullable = true)
 |-- level: string (nullable = true)
 |-- num_lectures: double (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- num_subscribers: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- published_timestamp: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- url: string (nullable = true)



In [177]:
business_finance.show(10)

+---------------+----------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|_corrupt_record|content_duration|course_id|        course_title|is_paid|             level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+---------------+----------------+---------+--------------------+-------+------------------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|           null|             1.5|  1070968|Ultimate Investme...|   True|        All Levels|        51.0|         23|           2147|200.0|2017-01-18T20:58:58Z|Business Finance|https://www.udemy...|
|           null|            39.0|  1113822|Complete GST Cour...|   True|        All Levels|       274.0|        923|           2792| 75.0|2017-03-09T16:34:20Z|Business Finance|https://www.udemy...|
|    

In [178]:
from pyspark.sql.functions import avg
from pyspark.sql.window import Window

In [170]:
# windowSpec  = Window.orderBy("num_subscribers")

In [171]:
# business_finance = business_finance.withColumn("avg_subscribers",avg("num_subscribers").over(windowSpec))

In [185]:
business_finance.agg({'price': 'mean','num_lectures': 'mean','num_reviews': 'mean','num_subscribers' : 'mean'}).show()

+-----------------+----------------+--------------------+-----------------+
|       avg(price)|avg(num_reviews)|avg(num_subscribers)|avg(num_lectures)|
+-----------------+----------------+--------------------+-----------------+
|68.55230125523012|63.5163179916318|  1563.7748953974894|32.43096234309623|
+-----------------+----------------+--------------------+-----------------+



In [186]:
business_finance = business_finance.select(business_finance.course_title,business_finance.subject)

In [187]:
business_finance.show()

+--------------------+----------------+
|        course_title|         subject|
+--------------------+----------------+
|Ultimate Investme...|Business Finance|
|Complete GST Cour...|Business Finance|
|Financial Modelin...|Business Finance|
|Beginner to Pro -...|Business Finance|
|How To Maximize Y...|Business Finance|
|Trading Penny Sto...|Business Finance|
|Investing And Tra...|Business Finance|
|Trading Stock Cha...|Business Finance|
|Options Trading 3...|Business Finance|
|The Only Investme...|Business Finance|
|Forex Trading Sec...|Business Finance|
|Trading Options W...|Business Finance|
|Financial Managem...|Business Finance|
|Forex Trading Cou...|Business Finance|
|Python Algo Tradi...|Business Finance|
|Short Selling: Le...|Business Finance|
|Basic Technical A...|Business Finance|
|The Complete Char...|Business Finance|
|7 Deadly Mistakes...|Business Finance|
|Financial Stateme...|Business Finance|
+--------------------+----------------+
only showing top 20 rows



In [188]:
business_finance.toPandas().to_csv('OutputDatas/3.Business_Finance.csv',index= False)

# 6. Find the courses which have more than 15 lectures.

In [189]:
more_than_15_lectures = df.select(df.course_title,df.num_lectures)

In [190]:
more_than_15_lectures.show(10)

+--------------------+------------+
|        course_title|num_lectures|
+--------------------+------------+
|Ultimate Investme...|          51|
|Complete GST Cour...|         274|
|Financial Modelin...|          51|
|Beginner to Pro -...|          36|
|How To Maximize Y...|          26|
|Trading Penny Sto...|          25|
|Investing And Tra...|          26|
|Trading Stock Cha...|          23|
|Options Trading 3...|          38|
|The Only Investme...|          15|
+--------------------+------------+
only showing top 10 rows



In [191]:
more_than_15_lectures.printSchema()

root
 |-- course_title: string (nullable = true)
 |-- num_lectures: string (nullable = true)



In [192]:
more_than_15_lectures = more_than_15_lectures.withColumn("num_lectures",col("num_lectures").cast("double"))

In [193]:
more_than_15_lectures.printSchema()

root
 |-- course_title: string (nullable = true)
 |-- num_lectures: double (nullable = true)



In [202]:
more_than_15_lectures = more_than_15_lectures.filter(more_than_15_lectures.num_lectures>15)

In [203]:
more_than_15_lectures.show()

+--------------------+------------+
|        course_title|num_lectures|
+--------------------+------------+
|Ultimate Investme...|        51.0|
|Complete GST Cour...|       274.0|
|Financial Modelin...|        51.0|
|Beginner to Pro -...|        36.0|
|How To Maximize Y...|        26.0|
|Trading Penny Sto...|        25.0|
|Investing And Tra...|        26.0|
|Trading Stock Cha...|        23.0|
|Options Trading 3...|        38.0|
|Forex Trading Sec...|        76.0|
|Trading Options W...|        17.0|
|Financial Managem...|        19.0|
|Forex Trading Cou...|        16.0|
|Python Algo Tradi...|        42.0|
|Short Selling: Le...|        19.0|
|Basic Technical A...|        16.0|
|The Complete Char...|        52.0|
|7 Deadly Mistakes...|        23.0|
|Winning Forex Tra...|        25.0|
|Forex Traders - C...|        39.0|
+--------------------+------------+
only showing top 20 rows



In [204]:
more_than_15_lectures.toPandas().to_csv('OutputDatas/6.More_than_15_lectures.csv',index= False)

# 7. List courses on the basis of level.

In [213]:
level_basis = df.orderBy(df.level)

In [214]:
level_basis.show()

+--------------------+----------------+---------+--------------------+-------+----------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|     _corrupt_record|content_duration|course_id|        course_title|is_paid|     level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+--------------------+----------------+---------+--------------------+-------+----------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|{"course_id": "52...|            null|     null|                null|   null|      null|        null|       null|           null| null|                null|            null|                null|
|                null|             4.0|   798740|Forex Traders - C...|   True|All Levels|          39|         84|           2295|  200|2016-05-02T19:26:48Z|Business Finance|https://www.udemy...|
|                nul

In [215]:
level_basis= level_basis.dropna(subset =['subject'])

In [216]:
level_basis.show()

+---------------+----------------+---------+--------------------+-------+----------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|_corrupt_record|content_duration|course_id|        course_title|is_paid|     level|num_lectures|num_reviews|num_subscribers|price| published_timestamp|         subject|                 url|
+---------------+----------------+---------+--------------------+-------+----------+------------+-----------+---------------+-----+--------------------+----------------+--------------------+
|           null|             4.0|   798740|Forex Traders - C...|   True|All Levels|          39|         84|           2295|  200|2016-05-02T19:26:48Z|Business Finance|https://www.udemy...|
|           null|            16.0|   301442|Black Algo Tradin...|   True|All Levels|         227|       1113|          20195|  200|2014-10-27T22:01:36Z|Business Finance|https://www.udemy...|
|           null|             2.0|   506568|C

In [217]:
level_basis = level_basis.select(level_basis.course_title,level_basis.level)

In [218]:
level_basis.show()

+--------------------+----------+
|        course_title|     level|
+--------------------+----------+
|Forex Traders - C...|All Levels|
|Black Algo Tradin...|All Levels|
|Create A Business...|All Levels|
|Complete GST Cour...|All Levels|
|The Complete Ethe...|All Levels|
|Python for Financ...|All Levels|
|Beginner to Pro i...|All Levels|
|Trading Stock Cha...|All Levels|
|Investing 101: Th...|All Levels|
|Forex Trading Sec...|All Levels|
|Learn MQL5: Build...|All Levels|
|Financial Managem...|All Levels|
|THE Forex Robot: ...|All Levels|
|Python Algo Tradi...|All Levels|
|The Complete Bitc...|All Levels|
|7 Deadly Mistakes...|All Levels|
|Mastering High Pr...|All Levels|
|Ultimate Investme...|All Levels|
|The Complete Shor...|All Levels|
|Trading Penny Sto...|All Levels|
+--------------------+----------+
only showing top 20 rows



In [220]:
level_basis.toPandas().to_csv('OutputDatas/7.level_basis_courses.csv',index= False)

# 8. Find the courses which have duration greater than 2 hours.


In [221]:
greater_than_2hours = df.select(df.course_title,df.content_duration)

In [222]:
greater_than_2hours.show()

+--------------------+------------------+
|        course_title|  content_duration|
+--------------------+------------------+
|Ultimate Investme...|               1.5|
|Complete GST Cour...|              39.0|
|Financial Modelin...|               2.5|
|Beginner to Pro -...|               3.0|
|How To Maximize Y...|               2.0|
|Trading Penny Sto...|               3.0|
|Investing And Tra...|               1.0|
|Trading Stock Cha...|               2.5|
|Options Trading 3...|               2.5|
|The Only Investme...|               1.0|
|Forex Trading Sec...|               5.0|
|Trading Options W...|               1.0|
|Financial Managem...|               1.5|
|Forex Trading Cou...|               2.0|
|Python Algo Tradi...|               7.0|
|Short Selling: Le...|               1.5|
|Basic Technical A...|               1.5|
|The Complete Char...|               4.0|
|7 Deadly Mistakes...|               1.5|
|Financial Stateme...|0.5833333333333334|
+--------------------+------------

In [223]:
greater_than_2hours.printSchema()

root
 |-- course_title: string (nullable = true)
 |-- content_duration: string (nullable = true)



In [224]:
greater_than_2hours = greater_than_2hours.withColumn("content_duration",col("content_duration").cast("float"))

In [225]:
greater_than_2hours.printSchema()

root
 |-- course_title: string (nullable = true)
 |-- content_duration: float (nullable = true)



In [226]:
greater_than_2hours= greater_than_2hours.filter(greater_than_2hours.content_duration> 2.0)

In [227]:
greater_than_2hours.show()

+--------------------+----------------+
|        course_title|content_duration|
+--------------------+----------------+
|Complete GST Cour...|            39.0|
|Financial Modelin...|             2.5|
|Beginner to Pro -...|             3.0|
|Trading Penny Sto...|             3.0|
|Trading Stock Cha...|             2.5|
|Options Trading 3...|             2.5|
|Forex Trading Sec...|             5.0|
|Python Algo Tradi...|             7.0|
|The Complete Char...|             4.0|
|Forex Traders - C...|             4.0|
|Beginner to Pro i...|             5.0|
|Investing 101: Th...|             4.5|
|Learn MQL5: Build...|             4.5|
|Forex Trading: Yo...|             3.0|
|Python Algo Stock...|             2.5|
|The Complete Bitc...|             6.5|
|Options Trading S...|            10.0|
|Forex Trading A-Z...|             5.5|
|Introduction to F...|             4.5|
|Beginner to Pro i...|             7.5|
+--------------------+----------------+
only showing top 20 rows



In [229]:
greater_than_2hours.toPandas().to_csv('OutputDatas/8.greater_than_2hours.csv',index= False)